<a href="https://colab.research.google.com/github/shinhs0920/Engineering-computing-application/blob/master/Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#seq2seq

##seq2seq란?

<br> seq2seq는 입력된 시계열 데이터를 다른 시계열 데이터로 변환하는 다양한 분야에서 사용되는 모델입니다. seq2seq2를 Encoder-Decoder 모델이라고도 합니다.
<br>대표적인 예 : 현대 챗봇(chatbot)과 기계 및 언어번역(Machine Translation) 
<br>- 입력 시퀀스와 출력 시퀀스를 각각 질문과 대답으로 구성하면 챗봇
<br>- 입력 시퀀스와 출력 시퀀스를 입력문장과 번역 문장으로 만들면 번역기로 만들 수 있다.

장점 : seq2seq2는 입력차원과 출력차원에서 자유롭습니다.
<br> -> 일반적인 딥러닝 모델의 경우 입력차원과 출력차원이 정의된 상태(제한된 상태)에서 학습 및 예측을 해야합니다.

###구조

<seq2seq2의 대표적인 예 - 언어번역 구조>

![image](https://user-images.githubusercontent.com/53015968/69904068-48eab000-13e5-11ea-9021-5ac19351ee5c.png)


기본적인 시퀸스-투-시퀸스 모델은 위 그림처럼 두개의 리커런트 뉴럴 네트워크(=RNN)으로 구성됩니다.<br> 또한 크게는 입력 데이터를 인코딩하는 인코더와 인코딩된 데이터를 디코딩하는 디코더로 이루어져 있습니다.

-> 인코딩(부호화) : 정보를 어떤 규칙에 따라 변환하는 것
<br> 디코딩(복호화) : 인코딩된 정보를 원래의 정보로 되돌리는 것

### 인코더(Encoder) - 입력 문장을 받는 RNN셀 입니다.

인코더는 RNN을 이용해 시계열 데이터을 은닉 상태 벡터(컨텍스트 벡터)로 변환합니다. 
<br>이 마지막 은닉 상태 벡터에 입력 문장을 번역하는데 필요한 정보가 인코딩 됩니다.

중요한 점은 은닉상태 벡터는 고정 길이 벡터 입니다.<br>따라서, 인코딩이라 함은 임의의 길이의 문장을 고정 길이 벡터로 변환하는 작업입니다.

입력 문장의 모든 단어들을 순차적으로 입력받은 뒤에 인코딩 후 다음 셀로 넘기고, 최종적으로 Encoder의 마지막 셀에 전달되어 모든 단어 정보들을 압축해서 하나의 벡터 [컨텍스트 벡터(context vector)]를 디코더로 전송합니다. 

![image](https://user-images.githubusercontent.com/53015968/69905015-a8e75380-13f1-11ea-9e6f-4b10e73bb267.png)



위 예시처럼 Encoder는 문장을 고정 길이 벡터로 변환합니다.

컨텍스트 벡터는 디코더 RNN 셀의 첫번째 은닉 상태로 사용됩니다.
<br> 각 셀은 긴 문장의 처리를 위해 LSTM셀로 구성됩니다.(GRU셀로 표현하기도 합니다)

<컨텍스트 벡터>

![image](https://user-images.githubusercontent.com/53015968/69903931-8bab8880-13e3-11ea-9355-7f2f06508a9f.png)


실제 모델에서는 수백개 이상의 차원을 가지고 있습니다.
<br>컨텍스트 벡터는 인코더에서의 마지막 RNN 셀의 은닉 상태값을 말하는 것이며, 이는 입력 문장의 모든 단어 토큰들의 정보를 요약해서 담고있다고 할 수 있습니다

정리하면 Encoder의 입력은 문장이고, 출력은 문장들을 인코딩한 hidden state 입니다.

### 디코더(Decoder) - 컨텍스트 벡터를 받아 한 개씩 순차적으로 출력합니다.

디코더는 기본적으로 RNNLM(RNN Language Model)입니다

디코더는 인코더의 마지막 RNN 셀의 은닉 상태인 컨텍스트 벡터를 첫번째 은닉 상태의 값으로 사용합니다.
<br>디코더의 첫번째 RNN 셀은 이 첫번째 은닉 상태의 값과, 현재 t에서의 입력값인 $<sos>$로부터, 다음에 등장할 단어를 예측합니다.

추가정보 : 위 예시에서는 $<sos>$라는 구분기호를 이용했습니다. 이는 Decoder에 문장 생성의 시작을 알리는 신호로 이용되며, $<eos>$를 출력할 때 까지 단어를 샘플링하도록 하기위한 종료신호 입니다.
<br> 구분기호를 $<go>$, 밑줄(_), **$<start>$** 등을 이용하기도 합니다.

그리고 이 예측된 단어는 다음 시점인 t+1 RNN에서의 입력값이 되고, 이 t+1에서의 RNN 또한 이 입력값과 t에서의 은닉 상태로부터 t+1에서의 출력 벡터. 즉, 또 다시 다음에 등장할 단어를 예측하게 될 것입니다.

### 단순 가변 길이 시계열 데이터 계산

seq2seq2에서 계산 중 다른 방법들과 다른 덧셈 계산에 대해 잠깐 알아보겠습니다.

덧셈을 문자(숫자)의 리스트로써 다루게 됩니다. 주의할 점은 '57+5' 또는 '628+521'에 대해 답('62',''1149)의 문자수가 문제마다 다릅니다. 
<br> '57+5'은 총 4문자이고 '628+521'은 총 7문자입니다.

따라서 가변 길이 시계열 데이터 계산을 미니배치로 학습하기 위해서는 패딩을 사용합니다. <br>패딩은 모든 데이터의 길이를 균일하게 맞추기 위한 기법입니다.
<br> 

![image](https://user-images.githubusercontent.com/53015968/69905322-5f990300-13f5-11ea-9433-cb1d41466505.png)


위의 예시처럼 데이터의 길이를 맞추고 남은 공간은 공백으로 남깁니다.<br>위 세가지의 덧셈 계산의 입력의 최대 문자수는 7 입니다. 덧셈 결과는 최대 4입니다.
<br>질문과 정답을 구분짓기 위하여 Decoder에 문자열을 생성하라는 신호인 구분자 밑줄(_)을 출력 앞에 붙입니다.

하지만 이는 원래는 존재하지 않던 패딩용 문자까지 seq2seq가 처리하게됩니다.<br>따라서, Decoder에 입력된 데이터가 패딩이라면 패딩용 문자가 손실의 결과에 반영하지 않도록 해야합니다.

그 방법은 softmax with loss 계층에 'mask'계층을 추가하면 됩니다
<br> mask -> 어디까지가 진짜 데이터고 어디까지가 더미 데이터인지를 구분하는 것으로써, 보통 0과 1로만 이루어진 같은 크기의 텐서이며 곱하기만 하면 불필요한 부분의 값은 모두 0이 되는 방식

seq2seq는 훈련 과정과 테스트 과정(또는 실제 번역기를 사람이 쓸 때)의 작동 방식이 조금 다릅니다

### seq2seq를 테스트 합니다.

<br>디코더는 초기 입력으로 문장의 시작을 의미하는 심볼 $<sos>$ (=start of sentence)가 들어갑니다. 디코더는 $<sos>$가 입력되면, 다음에 등장할 확률이 높은 단어를 예측합니다
<BR>첫번째 시점(time step)의 디코더 RNN 셀은 다음에 등장할 단어로 je를 예측하였습니다

첫번째 시점의 디코더 RNN 셀은 예측된 단어 je를 다음 시점의 RNN 셀의 입력으로 입력합니다. 그리고 두번째 시점의 디코더 RNN 셀은 입력된 단어 je로부터 다시 다음에 올 단어인 suis를 예측하고, 또 다시 이것을 다음 시점의 RNN 셀의 입력으로 보냅니다. 디코더는 이런 식으로 기본적으로 다음에 올 단어를 예측하고, 그 예측한 단어를 다음 시점의 RNN 셀의 입력으로 넣는 행위를 반복합니다. 이 행위는 문장의 끝을 의미하는 심볼인 <eos>가 다음 단어로 예측될 때까지 반복됩니다. 

테스트 과정에서는 앞서 설명한 과정과 같이 디코더는 오직 컨텍스트 벡터와 $<go>$만을 입력으로 받은 후에 다음에 올 단어를 예측하고, 그 단어를 다음 시점의 RNN 셀의 입력으로 넣는 행위를 반복합니다.

### seq2seq를 훈련합니다.

훈련 과정에서는 디코더에게 인코더가 보낸 컨텍스트 벡터와 실제 정답인 상황인 $<sos>$ je suis étudiant를 입력 받았을 때, je suis étudiant $<eos>$(=end of sentence)가 나와야 된다고 정답을 알려주면서 훈련합니다


### 임베딩 층(embedding layer)- 단어를 고정된 길이의 숫자 벡터로 임베딩합니다.

이번에는 입, 출력에 쓰이는 단어 토큰들이 있는 부분을 좀 더 확대해보겠습니다

아래 그림은 모든 단어에 대해 임베딩 과정을 거치게 해주는 embedding layer의 모습입니다.

![image](https://user-images.githubusercontent.com/53015968/69904173-e98d9f80-13e6-11ea-8bac-283f53be687b.png)


텍스트를 벡터로 바꾸는 방법으로 워드 임베딩이 사용됩니다. 즉, seq2seq에서 사용되는 모든 단어들은 워드 임베딩을 통해 임베딩 벡터로서 표현된 임베딩 벡터입니다.

예를 들어 아래의 그림과 같이 I, am, a, student 단어들은 위와 같이 단어를 고정된 길이의 실수 벡터로 임베딩 합니다.<br>실제 임베딩 벡터는 수백 개의 차원을 가질 수 있습니다.

![image](https://user-images.githubusercontent.com/53015968/69904212-602a9d00-13e7-11ea-8313-396fad6dd9e7.png)


### 디코더의 예측

이제 디코더가 다음에 등장할 단어를 예측하는 부분을 확대해보도록 하겠습니다.

![image](https://user-images.githubusercontent.com/53015968/69904264-f19a0f00-13e7-11ea-9df7-d3da21d09b50.png)


### 시퀀스-투-시퀀스 모델 - 코드

다음과 같은 모델을 예시로 seq2seq를 실행한다

![image](https://user-images.githubusercontent.com/53015968/69904553-1348c580-13eb-11ea-9df0-bf073f7b0856.png)


파일 : 구성 (표로 만들자)
seq2seq.py : 	시퀸스-투-시퀸스 모델을 생성하기 위한 라이브러리.
<br>translate/seq2seq_model.py : 뉴럴 번역 시퀸스-투-시퀸스 모델.
<br>translate/data_utils.py :	번역 데이타를 준비하기 위한 도움 함수들.
<br> translate/translate.py	: 번역 모델을 훈련하고 실행시키는 바이너리(binary).

기본적인 RNN 인코더-디코더 시퀸스-투-시퀸스 모델은 아래처럼 동작한다.

In [0]:
outputs, states = basic_rnn_seq2seq(encoder_inputs, decoder_inputs, cell)

 encoder_inputs : 인코더에 입력을 나타내는 텐서(tensor) 리스트이다. 
<br> -> 위 그림에서 A, B, C에 해당된다.
<br> decoder_inputs : 디코더에서 입력을 난타내는 텐서들이다.
<br> -> 위 그림에서 go, W , X, Y, Z, 이다
<BR> states : 모든 시간 단계에서 디코더의 내부 상태
<br> outputs : 각 시간 단계(time-step)에서의 디코더 결과
<br> -> 위 그림에서 W, X, Y, Z, eos

outputs과 states는 둘다 decoder_inputs와 동일한 크기의 텐서 리스트이다.

cell : models.rnn.rnn_cell.RNNCell 클래스의 인스턴스로써, 모델에서 어느 셀을 택할지 결정한다. (GRU 혹은 LSTM)
<br>rnn_cell : 여러층 셀을 만들고, 셀 입력과 결과에 드롭아웃(droout)을 추가한다.
<br> basic_rnn_seq2seq : 두개의 인수(outputs, states)를 return한다.

###embedding - RNN으로 구현하기

In [0]:
outputs, states = embedding_rnn_seq2seq(
    encoder_inputs, decoder_inputs, cell,
    num_encoder_symbols, num_decoder_symbols,
    output_projection=None, feed_previous=False)

embedding_rnn_seq2seq 모드에서 모든 입력들(encoder_inputs 과 decoder_inputs)은 이산 값(discrete value)(=변하는 양)을 나타내는 정수-텐서(integer-tensor)들 입니다. 그것들은 조밀한(dense) 표현으로 embedding 되어 질 것입니다.

임베딩을 만들기 위해서 나타나는 이산 심볼의 최대 수를 지정할 필요가 있습니다: 인코더에 num_encoder_symbols, 그리고 디코더에 num_decoder_symbols을 지정합니다. 

이것은 디코더에 전달되는 decoder_inputs를 사용할 것이다. feed_previous가 True로 설정되면, 디코더는 decoder_inputs의 단지 첫번째 원소만 사용할 것이다. 이 리스트에 모든 다른 텐서들은 무시 되어지고, 그대신 디코더의 이전 결과가 사용되어 질 것이다

feed_previous는 False로 설정했습니다.
<br>이는 디코더에 전달되는 decoder_inputs를 사용합니다. 
<br>만약, feed_previous가 True로 설정되면, 디코더는 decoder_inputs의 단지 첫번째 원소만 사용합니다. 이 결과는 리스트에 있는 모든 다른 텐서들은 무시 되어지고, 그대신 디코더의 이전 결과가 사용되어 질 것입니다.. 

사용된 중요한 인수는  output_projection 입니다.
<br>output_projection은 히든 노드에서 도출된 히든 스테이트 벡터를 vocab_size <br>차원의 sparse 벡터로 선형변환해주는 가중치 W와 bias term b로 구성된 튜플입니다. 그것이 명시되지 않는다면, 임베딩 모델의 결과는 각 생성된 심볼에 대해 로짓(logit)들로 나타내기 때문에 배치사이즈 x num_decoder_symbols의 형태를 가진 텐서가 될 것입니다. 
<br>이번 프로젝트를 위해 embedding_attention_seq2seq를 사용할 것입니다..